In [10]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [11]:
!mkdir tmp

In [12]:
# Data Load
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
num_hidden = 20
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
hidden = tf.layers.dense(x, units=num_hidden, activation=tf.nn.sigmoid, use_bias=True)
logits = tf.layers.dense(hidden, units=10, activation=None, use_bias=True)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(tf.nn.softmax(logits),1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
batch_size = 128
lr = 0.01
MaxEpochs = 5
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [16]:
# Checkpoint
saver = tf.train.Saver(max_to_keep=25)

# Train
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(MaxEpochs):
    for step in range(len(mnist.train.images) // batch_size + 1):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={x:batch_xs, y:batch_ys})
        if step % 200 == 0:
            curr_loss = sess.run(loss, feed_dict={x:mnist.train.images, y:mnist.train.labels})
            curr_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
            print(epoch, step, curr_loss, curr_acc)
            saver.save(sess, "./tmp/model_epoch{0}_step_{1}.ckpt".format(epoch, step))
    print("")

0 0 2.25157 0.2129
0 200 0.310286 0.9162
0 400 0.245774 0.928

1 0 0.237042 0.9312
1 200 0.219713 0.9319
1 400 0.192961 0.9361

2 0 0.190051 0.9379
2 200 0.199862 0.9309
2 400 0.170006 0.9392

3 0 0.171284 0.9409
3 200 0.159841 0.9428
3 400 0.155364 0.9419

4 0 0.146638 0.9444
4 200 0.143472 0.9455
4 400 0.138794 0.9454



In [17]:
saver.restore(sess, 'tmp/model_epoch2_step_200.ckpt')
curr_loss = sess.run(loss, feed_dict={x:mnist.train.images, y:mnist.train.labels})
curr_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
print(curr_loss, curr_acc)

INFO:tensorflow:Restoring parameters from tmp/model_epoch2_step_200.ckpt
0.199862 0.9309
